In [9]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

import warnings
# Ignorar os warnings
warnings.filterwarnings("ignore")

import time

In [10]:
def create_windowed_dataset(sequencia_original, window_size):
    T = len(sequencia_original)
    L = window_size

    janelas = []

    for i in range(T - L + 1):
        ith_janela = pd.DataFrame()

        for j in range(i+L-1, i-1, -1):
            jth_amostra = sequencia_original.iloc[[j]]
            jth_amostra.columns = [
                f'x_acc{j-i}',
                f'y_acc{j-i}',
                f'z_acc{j-i}',
                f'x_grav{j-i}',
                f'y_grav{j-i}',
                f'z_grav{j-i}',
                f'x_gyro{j-i}',
                f'y_gyro{j-i}',
                f'z_gyro{j-i}',
                f'x_accl{j-i}',
                f'y_accl{j-i}',
                f'z_accl{j-i}',
                f'x_magn{j-i}',
                f'y_magn{j-i}',
                f'z_magn{j-i}',
                'label'
            ]
            jth_amostra = jth_amostra.reset_index(drop=True)

            if j == i+L-1:
                label = jth_amostra['label']

            jth_amostra = jth_amostra.drop('label', axis=1)
            ith_janela[jth_amostra.columns] = jth_amostra

        ith_janela['label'] = label

        janelas.append(ith_janela)

    new_df = pd.concat(janelas)
    new_df.index = range(len(new_df.index))

    return new_df

In [11]:

def process_data(train, windows):
    start_process = time.time()
    # Salvar o novo dataset com as janelas


    train_df = create_windowed_dataset(train, windows)
    
    # scaler = StandardScaler()

    # # Normalização
    # cols = train_df.columns
    # sc_train = scaler.fit_transform(train_df.values)
    # sc_test = scaler.transform(test_df.values)
    # sc_validacao = scaler.transform(validacao_df.values)

    # # Salvar a normalização
    # train_df = pd.DataFrame(sc_train, columns=cols)
    # test_df = pd.DataFrame(sc_test, columns=cols)
    # validacao_df = pd.DataFrame(sc_validacao, columns=cols)

    # # Teste
    # n = round(len(train_df) * 0.30)
    # add = train_df.tail(n)  # 30%
    # # Apagar as linhas do df de treino que foram utilizadas no teste
    # train_df.drop(add.index, inplace=True)
    
    # test_df = test_df.append(add, ignore_index=True)
    

    # #Validação
    # n = round(len(train_df) * 0.20)
    # add2 = train_df.tail(n) #20%

    # #TODO: nao adicionar o teste no conjunto de validacao, pegar de um usuario de fora
    # val_df = pd.concat([add2, validacao_df], ignore_index=True)
    # # Apaga as linhas do df de treino que foram utilizadas no teste
    # train_df.drop(add2.index, inplace=True)
    
    #TODO: FIM

    
    end_process = time.time()
    print("Tempo de execução do processamento: ", end_process - start_process)
    time_windows.append(end_process - start_process) 

    return train_df



In [12]:
import os
output_dir = 'testes'  # Nome da pasta de destino

# Verifica se o diretório de destino existe, caso contrário, cria-o
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [13]:
# # Lista de usuarios
users = [
    'enrolment_01',
    'enrolment_02',
    'enrolment_03',
    'enrolment_04',
    'enrolment_05',
    'enrolment_06',
    'enrolment_07',
    'enrolment_08',
    'enrolment_09',
    'enrolment_10',
    'enrolment_11',
    'enrolment_12',
    'enrolment_13',
    'enrolment_14',
    'enrolment_15',
    'enrolment_16',
    'enrolment_17',
    'enrolment_18',
    'enrolment_19',
    'enrolment_20'
]

# Lista de tamanhos de janela
window_size_list = [3]
time_windows = []
for user in users:

    genuine = pd.read_csv('Comparacoes/Milisegundos/' + user + '/random/user.csv')
    #impostor = pd.read_csv('Comparacoes/Milisegundos/' + user + '/random/verification2.csv')
# validacao = pd.read_csv('user_validacao.csv')

# # Preencher valores faltantes com 0
    genuine = genuine.fillna(0)
    #impostor = impostor.fillna(0)
# validacao = validacao.fillna(0)

# # Acrescentar Labels
    genuine['label'] = 0
    #impostor['label'] = 1
# validacao['label'] = 1

# # Remover colunas ascii_code_touch e timestamp
    genuine = genuine.drop(['ascii_code', 'timestamp'], axis=1)
    #impostor = impostor.drop(['ascii_code', 'timestamp'], axis=1)
# validacao = validacao.drop(['ascii_code_touch', 'timestamp'], axis=1)

    for window_size in window_size_list:
        # TODO: salvar esses conjuntos para cada window_size para cada usuario em disco para nao realizar esse processamento a cada execucao

        print(f"Usuário: {user}, tamanho de janela: {window_size}")

        train_df = process_data(genuine, window_size)

        # Salvando o dataframe em um arquivo CSV
        train_df.to_csv(os.path.join(output_dir, f'{user}_train_data_window_{window_size}.csv'), index=False)
        #val_df.to_csv(os.path.join(output_dir, f'val_data_window_{window_size}.csv'), index=False)
        #test_df.to_csv(os.path.join(output_dir, f'{user}_test_data_window_{window_size}.csv'), index=False)


Usuário: enrolment_01, tamanho de janela: 3
Tempo de execução do processamento:  1.5355968475341797
Usuário: enrolment_02, tamanho de janela: 3
Tempo de execução do processamento:  5.212859869003296
Usuário: enrolment_03, tamanho de janela: 3
Tempo de execução do processamento:  2.4212913513183594
Usuário: enrolment_04, tamanho de janela: 3
Tempo de execução do processamento:  2.2936084270477295
Usuário: enrolment_05, tamanho de janela: 3
Tempo de execução do processamento:  1.867447853088379
Usuário: enrolment_06, tamanho de janela: 3
Tempo de execução do processamento:  2.47928786277771
Usuário: enrolment_07, tamanho de janela: 3
Tempo de execução do processamento:  5.143961429595947
Usuário: enrolment_08, tamanho de janela: 3
Tempo de execução do processamento:  2.0159661769866943
Usuário: enrolment_09, tamanho de janela: 3
Tempo de execução do processamento:  2.397937297821045
Usuário: enrolment_10, tamanho de janela: 3
Tempo de execução do processamento:  2.7168149948120117
Usuári

In [14]:
genuine.shape

(592, 16)

In [15]:
print(np.mean(time_windows))

3.0161159515380858


In [16]:
# df = pd.read_csv("testes/enrolment_04_train_data_window_2.csv")

# df

janela 2 = 2.009864628314972
janela 3 = 4.52887316942215
janela 4 = 4.005613911151886
janela 5 = 7.212210655212402
janela 6 = 6.485075569152832
janela 7 = 33.1065772652626